# 1. Imports and File selection 

In [ ]:
import io
import ipywidgets as widgets
import math
import numpy
import psycopg
import pandas as pd
import requests
import sqlite3
import sys
import tqdm
import warnings

from config import load_config
from ipyfilechooser import FileChooser
from scipy import stats
from scipy.stats import ttest_ind
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import insert
from sqlite3 import Error
from sqlite3 import IntegrityError

## Select Baseline .csv File

In [ ]:
starting_directory = '/Volumes'
baseline_chooser = FileChooser(starting_directory)
display(baseline_chooser)

## Select Tap .csv File

In [ ]:
tap_chooser=FileChooser('/Volumes')
display(tap_chooser)

## Select Post Stimulus Arousal .csv File

In [ ]:
psa_chooser = FileChooser('/Volumes')
display(psa_chooser)

In [ ]:
screens = ['PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 
           'Neuron_Genes_Screen', 'PD_GWAS_Locus71_Screen', 'ASD_WGS_Screen', 'Miscellaneous']

screen_chooser = widgets.Select(options=screens, value=screens[0], description='Screen:')
display(screen_chooser)

In [ ]:
Screen=screen_chooser.value
folder_path=baseline_chooser.selected_path
print(folder_path)

## Read baseline, tap and post stimulus arousal (psa) data

In [ ]:
# Read the baseline file
baseline_output = pd.read_csv(baseline_chooser.selected, index_col=0)#.drop(columns=['index'])

print(f"\nShape of the baseline .csv file: {baseline_output.shape}")

# Print the first five rows of the file
baseline_output.head()

In [ ]:
# Read the tap file
tap_output = pd.read_csv(tap_chooser.selected, index_col=0)

print(f"\nShape of the psa .csv file: {tap_output.shape}")

# Print the first five rows of the file
tap_output.head()

In [ ]:
# Read the psa file
psa_output = pd.read_csv(psa_chooser.selected, index_col=0)

for cols in ['Instantaneous Speed', 'Interval Speed',
       'Bias', 'Morphwidth', 'Midline', 'Area', 'Angular Speed',
       'Aspect Ratio', 'Kink', 'Curve', 'Crab', 'Pathlength']:
    psa_output.rename(columns={cols: f"PSA {cols}"}, inplace=True)

print(f"\nShape of the tap .csv file: {psa_output.shape}")

# Print the first five rows of the file
psa_output.head()

### Merge PSA with Tap response

In [ ]:
tap_psa_output = pd.merge(
    tap_output, psa_output.drop(columns=['Experiment', 'Time', 'Tap', 'PSA Morphwidth', 
                                         'PSA Midline', 'PSA Area', 'PSA Angular Speed',]),
    how='outer', 
    on=['dataset', 'Gene', 'Allele', 'Date', 'Plate_id', 'Screen', "taps" ] 
)

tap_psa_output = tap_psa_output[['dataset', 'Gene', 'Allele', 'Date', 'Plate_id', 'plate', 
                                 'Screen', 'taps', 'time', 'dura', 'dist', 'prob', 'speed',
                                 'PSA Instantaneous Speed', 'PSA Interval Speed', 'PSA Bias',
                                 'PSA Aspect Ratio', 'PSA Kink', 'PSA Curve', 'PSA Crab'
                                 ]]

print(f"Shape of the dataframe: {tap_psa_output.shape}")

tap_psa_output.rename(columns={
    'prob': 'Probability',
    'dura': 'Duration',
    'speed': 'Speed'
}, inplace=True)

tap_psa_output.head()

In [ ]:
# tap_psa_output.to_csv("tap_psa_output.csv")

# 2. DataFrame preparation

### 2.1. Tap Data

In [ ]:
# Dataframe for first tap
PD_first_tap = (
    tap_output[(tap_output.taps==1)]
    .reset_index().drop(columns="index")
    .rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
)

PD_first_tap.head()

In [ ]:
# Dataframe for recovery taps
PD_recov_taps = (
    tap_output[(tap_output.taps==31)]
    .reset_index().drop(columns="index")
    .rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
)

PD_recov_taps.head()

In [ ]:
# Dataframe for last three taps
PD_final_taps = (
    tap_output[((tap_output.taps >= 28) & (tap_output.taps <= 30))]
    .groupby(["dataset", "Date","Plate_id","Screen","Gene","Allele","plate"])
    .mean()
    .reset_index()
    .rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")
)

PD_final_taps.head()

In [ ]:
# Dataframe to analyse habituation behaviour after merging first tap and final taps

PD_habit_levels = pd.merge(
    PD_first_tap, 
    PD_final_taps, 
    on =['dataset', 'plate', "Plate_id", "Screen", "Gene", "Allele", "Date"], how ='left'
).drop(columns=['time_x','time_y','dist_x','dist_y', 'taps_x', 'taps_y']).dropna()

PD_habit_levels['habit_dura'] = PD_habit_levels['init_dura'] - PD_habit_levels['final_dura']

PD_habit_levels['habit_prob'] = PD_habit_levels['init_prob'] - PD_habit_levels['final_prob']

PD_habit_levels['habit_speed'] = PD_habit_levels['init_speed'] - PD_habit_levels['final_speed']

In [ ]:
# Continue to analyse habituation behaviour after merging with recovery taps

if PD_recov_taps.empty:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='outer')
else:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')

if Screen not in ['Neuron_Genes_Screen', 'G-Proteins_Screen']:
    PD_habituation = PD_habituation.dropna() 

PD_habituation['recovery_dura']=(PD_habituation.recov_dura-PD_habituation.init_dura)/PD_habituation.init_dura*100

PD_habituation['recovery_prob']=(PD_habituation.recov_prob-PD_habituation.init_prob)/PD_habituation.init_prob*100

PD_habituation['recovery_speed']=(PD_habituation.recov_speed-PD_habituation.init_speed)/PD_habituation.init_speed*100

PD_habituation['memory_retention_dura']=(PD_habituation.recov_dura-PD_habituation.final_dura)

PD_habituation['memory_retention_prob']=(PD_habituation.recov_prob-PD_habituation.final_prob)

PD_habituation['memory_retention_speed']=(PD_habituation.recov_speed-PD_habituation.final_speed)


# Rename `PD_habituation` to `tap_data` based on the condition below
if Screen in ['Neuron_Genes_Screen', 'G-Proteins_Screen']:
    tap_data=PD_habituation.dropna(subset = ['init_dura', 'init_prob', 'init_speed', 'plate', 'Date', 'Plate_id',
       'Screen', 'dataset', 'Gene', 'Allele', 'final_dura', 'final_prob',
       'final_speed', 'habit_dura', 'habit_prob', 'habit_speed'])
else:
    tap_data=PD_habituation.dropna() 


# Display final dataframe
tap_data.head()


### 2.2. PSA data

In [ ]:
# function to calculate Initial, Final, Peak, ect values for specified column (metric)

def summary_metrics(df, metric = 'Instantaneous Speed'):

    initial = df[metric].iloc[0]
    recovery = df[metric].iloc[-1]
    final = df[metric].iloc[-4:-1].mean() # average of the last three taps
    peak = df[metric].max()
    peak_id = df[metric].values.argmax() # only used for peak tap calculation
    peak_tap = df['taps'].iloc[peak_id]
    mean = df[metric].mean()
    sensitization = peak - initial
    habituation = peak - final 
    spontaneous_recovery = 100*(initial - recovery)/initial if metric not in ['PSA Aspect Ratio', 'PSA Kink', 'PSA Curve', 'PSA Crab'] else 100*(recovery - initial)/initial
    memory_retention = final - recovery if metric not in ['PSA Aspect Ratio', 'PSA Kink', 'PSA Curve', 'PSA Crab'] else recovery - final
    # initial_to_peak = df[metric].iloc[: peak_id+1].mean()
    # peak_to_recovery = df[metric].iloc[peak_id:].mean()

    

    return pd.Series({
        f'Initial {metric}': initial, 
        f'Final {metric}': final,
        f'Recovery {metric}': recovery, 
        f'Peak {metric}': peak,
        f'Peak Tap Number {metric}': peak_tap,
        f'Average {metric}': mean,
        f'Sensitization {metric}': sensitization,
        f'Habituation {metric}': habituation,
        f'Spontaneous Recovery {metric}': spontaneous_recovery,
        f'Memory Retention {metric}': memory_retention
        # f'Initial_to_peak {metric}': initial_to_peak, 
        # f'Peak_to_recovery {metric}': peak_to_recovery
        })

In [ ]:
warnings.filterwarnings('ignore')

# columns to summarize
metrics_to_summarize = ['PSA Instantaneous Speed', 'PSA Bias', 'PSA Angular Speed', 
                        'PSA Aspect Ratio', 'PSA Kink', 'PSA Curve', 'PSA Crab']

# standard columns
group_cols = ['Experiment', 'Plate_id', 'Date', 'Screen', 'dataset', 'Gene', 'Allele']

# pass each column to summarise through `summary_metrics` function and merge the summarised values to psa_output
psa_data = psa_output[group_cols].drop_duplicates()
for metric in metrics_to_summarize:
    summary = psa_output.groupby(group_cols).apply(lambda x: summary_metrics(x, metric)).reset_index()
    psa_data = pd.merge(psa_data, summary, on=group_cols, how='left')

In [ ]:
psa_data.head()

In [ ]:
psa_data.columns

# 3. Run Statistics (T-Test and mean sample distance) on Data

## 3.1 Generate dataframes conditioned by `baseline` (True/False) and `allele` (True/False)

In [ ]:
def get_output_byplate(output, baseline=["true", "false", "psa"], allele = [False, True]):
    """
    Aggregates data by 'Gene' or 'Allele' and drops 'Plate_id','Date','Screen','dataset', etc

    Parameters:
        output (pd.DataFrame): Input DataFrame (either baseline_output or tap_data)
        baseline (boolean): whether data is baseline (True) or tap response (False)
        allele (boolean): group by allele (True) or group by gene (False)

    Returns:
        A DataFrame with plate-level averages
    """
    
    # columns to delete if baseline = true
    if baseline == "true":
        drop_col = ['Plate_id','n','Number','Time','Screen','Date','Allele']
    # columns to delete if baseline = false
    elif baseline == "false":
        drop_col = ['Plate_id','Screen','Date','Allele','dist','plate','time',
                       'taps','recov_dura','recov_prob','recov_speed']
    # columns to delete if baseline = psa
    else: 
        drop_col = ['Experiment', 'Plate_id', 'Date', 'Screen', 'Allele']

    drop_col.append('Gene') if allele else drop_col.append('dataset')
     
    output_byplate = output.groupby(
        by=['Plate_id','Date','Screen','dataset','Gene','Allele'],
        as_index=False).mean().drop(columns=drop_col)
    
    return output_byplate

#### 3.1.1 `baseline` = True, `allele` = False

In [ ]:
baseline_output_byplate=get_output_byplate(baseline_output, baseline= "true", allele=False)

print(f"Shape: {baseline_output_byplate.shape}")

baseline_output_byplate.head()

#### 3.1.2 `baseline` = False, `allele` = False

In [ ]:
tap_data_byplate=get_output_byplate(tap_data, baseline="false", allele=False)

print(f"Shape: {tap_data_byplate.shape}")

tap_data_byplate.head()

#### 3.1.3 `baseline` = True, `allele` = True

In [ ]:
baseline_output_allele_byplate = get_output_byplate(baseline_output,baseline="true", allele=True)

print(f"Shape: {baseline_output_allele_byplate.shape}")

baseline_output_allele_byplate.head()

#### 3.1.4 `baseline` = False, `allele` = True

In [ ]:
tap_data_allele_byplate = get_output_byplate(tap_data, baseline="false", allele=True)

print(f"Shape: {tap_data_allele_byplate.shape}")

tap_data_allele_byplate.head()

In [ ]:
# tap_data_allele_byplate[tap_data_allele_byplate.dataset=='N2_XJ1']

#### 3.1.5 `baseline` = "psa" , `allele` = False

In [ ]:
psa_data_byplate = get_output_byplate(psa_data, baseline="psa", allele=False)

print(f"Shape: {psa_data_byplate.shape}")

psa_data_byplate.head()

#### 3.1.6 `baseline` = "psa" , `allele` = True

In [ ]:
psa_data_allele_byplate = get_output_byplate(psa_data, baseline="psa", allele=True)

print(f"Shape: {psa_data_allele_byplate.shape}")

psa_data_allele_byplate.head()

## 3.2 Calculate Mean Distances and CIs

In [ ]:

def extract_phenotypes(df):
    ''' 
    Splits a multi-column DataFrame into a list of DataFrames, each containing one phenotype

    input: 
        df (pd.DataFrame): dataframe with multiple columns (1st column is the index, the other are phenotypes)

    returns:
        list_phenotypes_df: list with 2 columns - one for index and one for phenotype, 
            for how many phenotypes there are in the input
    '''
    list_phenotypes_df = []
    index = df.columns[0]
    for i in df.columns[1:]:
        list_phenotypes_df.append(df[[index, i]].copy())

    return list_phenotypes_df



def ci95(df):
    """
    input: df of 4 columns: index, mean, count, std

    returns: df of 6 columns: index, mean, count, std, ci95_hi, ci95_low

    """
    for metric in df.columns.levels[0]:
        if metric == 'Gene':
            pass
        else:
            ci95_hi = []
            ci95_lo = []
            for i in df[metric].index:
                m = df[metric]['mean'].loc[i]
                c = df[metric]['count'].loc[i]
                s = df[metric]['sem'].loc[i]
                ci95_hi.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[1])
                ci95_lo.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[0])
            df[metric,'ci95_hi'] = ci95_hi
            df[metric,'ci95_lo'] = ci95_lo
            # df[metric,'ci95']=list(zip(ci95_lo,ci95_hi))
            
    return df



def calculate_MSD(list_of_dfs, by):
    new_list_of_dfs = []
    
    for df in list_of_dfs:
        # Get phenotype column name (assuming 2nd column is the metric)
        pheno_col = df.columns[1]
        
        # Calculate statistics
        stats = df.groupby(by)[df.columns[1]].agg(['mean', 'count', 'sem'])

        
        # Convert to MultiIndex if needed (more robust version)
        if not isinstance(stats.columns, pd.MultiIndex):
            stats.columns = pd.MultiIndex.from_tuples([(pheno_col, col) for col in stats.columns])
        
        # Calculate CI
        stats_2 = ci95(stats)
        
        # Get N2 control data
        if Screen == "Neuron_Genes_Screen":
            N2_mask = stats_2.index == 'N2' if by == "Gene" else stats_2.index.isin(['N2_XJ1','N2_N2'])
        else:
            N2_mask = stats_2.index == 'N2'
            
        N2_data = stats_2[N2_mask]
        
        # Subtract N2 values
        stats_2.iloc[:, 0] -= N2_data.iloc[0, 0]  # mean
        stats_2.iloc[:, 3] -= N2_data.iloc[0, 0]  # ci95_hi
        stats_2.iloc[:, 4] -= N2_data.iloc[0, 0]  # ci95_low
        
        new_list_of_dfs.append(stats_2)
    
    return new_list_of_dfs

In [ ]:
def calculate_MSD(list_of_dfs, by):
    new_list_of_dfs = []
    
    for df in list_of_dfs:
        # Get phenotype column name (assuming 2nd column is the metric)
        pheno_col = df.columns[1]
        
        # Create proper MultiIndex structure
        stats = df.groupby(by)[df.columns[1]].agg(['mean', 'count', 'sem'])

        # Convert to MultiIndex if needed (more robust version)
        if not isinstance(stats.columns, pd.MultiIndex):
            stats.columns = pd.MultiIndex.from_tuples([(pheno_col, col) for col in stats.columns])
        
        # Calculate CIs
        stats_2 = ci95(stats)
        
        # Get N2 control data
        if Screen == "Neuron_Genes_Screen":
            N2_mask = stats_2.index == 'N2' if by == "Gene" else stats_2.index.isin(['N2_XJ1','N2_N2'])
        else:
            N2_mask = stats_2.index == 'N2'
            
        N2_data = stats_2[N2_mask]
        
        # Subtract N2 values
        stats_2.iloc[:, 0] -= N2_data.iloc[0, 0]  # mean
        stats_2.iloc[:, 3] -= N2_data.iloc[0, 0]  # ci95_hi
        stats_2.iloc[:, 4] -= N2_data.iloc[0, 0]  # ci95_low
        
        new_list_of_dfs.append(stats_2)
    
    return new_list_of_dfs

In [ ]:
def get_MSD(list_MSD):
    '''
    input: List of dataframes, each representing a phenotype with calculated MSD.

    returns: Single combined dataframe joining all input dataframes with MSD values.
    '''
    for a in list_MSD:
        if a.columns.levels[0] == list_MSD[0].columns.levels[0]:
            MSD=a
        else:
            MSD=MSD.join(a)
    return MSD

In [ ]:
def get_combined_MSD(baseline_byplate,tap_byplate, psa_byplate, by=['Gene','dataset']):
    """
    Combines MSD datafram from baseline plates and tap plates

    input:
        - baseline_byplate: baseline data by plate
        - tap_byplate: tap data by plate
        - by: what to group by "Gene" or "dataset"
    returns:
        - combined MSD dataframe
    """
    list_baseline_MSD=calculate_MSD(extract_phenotypes(baseline_byplate), by=by)

    list_tap_MSD=calculate_MSD(extract_phenotypes(tap_byplate), by=by)

    list_psa_MSD=calculate_MSD(extract_phenotypes(psa_byplate), by=by)

    baseline_MSD = get_MSD(list_baseline_MSD)
    
    tap_MSD = get_MSD(list_tap_MSD)

    psa_MSD = get_MSD(list_psa_MSD)

    combined_MSD = pd.merge(pd.merge(baseline_MSD, tap_MSD, on=by, how='outer'), psa_MSD, on=by, how='outer')

    combined_MSD=combined_MSD.rename(columns={"habit_dura":"Habituation of Response Duration",
                                         "habit_prob": "Habituation of Respones Probability",
                                         "habit_speed":"Habituation of Response Speed",
                                         "init_dura": "Initial Response Duration",
                                         "init_prob": "Initial Response Probability",
                                         "init_speed": "Initial Response Speed",
                                         "final_dura": "Final Response Duration",
                                         "final_prob": "Final Response Probability",
                                         "final_speed": "Final Response Speed",
                                         "recovery_dura": "Spontaneous Recovery of Response Duration",
                                         "recovery_prob": "Spontaneous Recovery of Response Probability",
                                         "recovery_speed": "Spontaneous Recovery of Response Speed",
                                         "memory_retention_dura": "Memory Retention of Response Duration",
                                         "memory_retention_prob": "Memory Retention of Response Probability",
                                         "memory_retention_speed": "Memory Retention of Response Speed"})

    combined_MSD=combined_MSD.reset_index()
    combined_MSD.columns = combined_MSD.columns.to_flat_index().str.join('-')
    combined_MSD=combined_MSD.rename(columns={by+"-": by})
    combined_MSD['Screen']=Screen
    
    return combined_MSD

### 3.2.1 Gene-level SMD

In [ ]:
combined_MSD=get_combined_MSD(baseline_output_byplate,
                              tap_data_byplate, 
                              psa_data_byplate,
                              by='Gene')

combined_MSD.head()

### 3.2.2 Allele-level SMD

In [ ]:
allele_combined_MSD=get_combined_MSD(baseline_output_allele_byplate,
                                     tap_data_allele_byplate, 
                                     psa_data_allele_byplate,
                                     by='dataset')

allele_combined_MSD.head()

## 3.3 T-Stat analysis

In [ ]:
def baseline_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframe and list of metrics for baseline analysis

    input:
        by (list): what to group by "Gene" or "dataset"
        
    returns:
        list_baseline_Tstats: dataframes to store t-statistics
        list_baseline_metrics: dataframes to store metic names
    """
    PD_baseline_instantspeed_T=pd.DataFrame(columns = [by,"Instantaneous Speed"])
    PD_baseline_intspeed_T=pd.DataFrame(columns = [by,"Interval Speed"])
    PD_baseline_bias_T=pd.DataFrame(columns = [by,"Bias"])
    PD_baseline_morphwidth_T=pd.DataFrame(columns = [by,"Morphwidth"])
    PD_baseline_midline_T=pd.DataFrame(columns = [by,"Midline"])
    PD_baseline_area_T=pd.DataFrame(columns = [by,"Area"])
    PD_baseline_angularspeed_T=pd.DataFrame(columns = [by,"Angular Speed"])
    PD_baseline_aspectratio_T=pd.DataFrame(columns = [by,"Aspect Ratio"])
    PD_baseline_kink_T=pd.DataFrame(columns = [by,"Kink"])
    PD_baseline_curve_T=pd.DataFrame(columns = [by,"Curve"])
    PD_baseline_crab_T=pd.DataFrame(columns = [by,"Crab"])
    PD_baseline_pathlength_T=pd.DataFrame(columns = [by,"Pathlength"])

    list_baseline_Tstats=[PD_baseline_instantspeed_T,
                        PD_baseline_intspeed_T,
                        PD_baseline_bias_T,
                        PD_baseline_morphwidth_T,
                        PD_baseline_midline_T,
                        PD_baseline_area_T,
                        PD_baseline_angularspeed_T,
                        PD_baseline_aspectratio_T,
                        PD_baseline_kink_T,
                        PD_baseline_curve_T,
                        PD_baseline_crab_T,
                        PD_baseline_pathlength_T]

    list_baseline_metrics=["Instantaneous Speed",
                        "Interval Speed",
                        "Bias",
                        "Morphwidth",
                        "Midline",
                        "Area",
                        "Angular Speed",
                        "Aspect Ratio",
                        "Kink",
                        "Curve",
                        "Crab",
                        "Pathlength"]
    
    return list_baseline_Tstats, list_baseline_metrics

In [ ]:
def tap_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframes and list of metrics for tap analysis

    input:
        by (list): what to group by "Gene" or "dataset"
        
    returns:
        list_tap_Tstats: dataframes to store t-statistics
        list_tap_metrics: dataframes to store metic names
    """
    recovery_dura=pd.DataFrame(columns = [by,"Recovery Duration"])
    recovery_prob=pd.DataFrame(columns = [by,"Recovery Probability"])
    recovery_speed=pd.DataFrame(columns = [by,"Recovery Speed"])
    memory_retention_dura=pd.DataFrame(columns = [by,"Memory Retention Duration"])
    memory_retention_prob=pd.DataFrame(columns = [by,"Memory Retention Probability"])
    memory_retention_speed=pd.DataFrame(columns = [by,"Memory Retention Speed"])
    init_dura=pd.DataFrame(columns = [by,"Initial Duration"])
    init_prob=pd.DataFrame(columns = [by,"Initial Probability"])
    init_speed=pd.DataFrame(columns = [by,"Initial Speed"])
    final_dura=pd.DataFrame(columns = [by,"Final Duration"])
    final_prob=pd.DataFrame(columns = [by,"Final Probability"])
    final_speed=pd.DataFrame(columns = [by,"Final Speed"])
    hab_dura=pd.DataFrame(columns = [by,"Habituation of Duration"])
    hab_prob=pd.DataFrame(columns = [by,"Habituation of Probability"])
    hab_speed=pd.DataFrame(columns = [by,"Habituation of Speed"])

    list_tap_Tstats = [recovery_dura,
                    recovery_prob,
                    recovery_speed,
                    memory_retention_dura,
                    memory_retention_prob,
                    memory_retention_speed,
                    init_dura,
                    init_prob,
                    init_speed,
                    final_dura,
                    final_prob,
                    final_speed,
                    hab_dura,
                    hab_prob,
                    hab_speed]
    
    list_tap_metrics = ["recovery_dura",
                        "recovery_prob",
                        "recovery_speed",
                        "memory_retention_dura",
                        "memory_retention_prob",
                        "memory_retention_speed",
                        "init_dura",
                        "init_prob",
                        "init_speed",
                        "final_dura",
                        "final_prob",
                        "final_speed",
                        "habit_dura",
                        "habit_prob",
                        "habit_speed"]
    
    return list_tap_Tstats, list_tap_metrics

In [ ]:
def psa_metrics(by=["Gene", "dataset"]):
    """
    Create a list of empty dataframes and list of metric names for PSA summary analysis.

    input:
        by (list): what to group by ("Gene" or "dataset")

    returns:
        list_psa_Tstats: list of empty DataFrames for t-statistics
        list_psa_metrics: list of metric names (short strings)
    """


    psa_initial_speed = pd.DataFrame(columns=[by,"Initial PSA Instantaneous Speed"])
    psa_final_speed = pd.DataFrame(columns=[by,"Final PSA Instantaneous Speed"])
    psa_recovery_speed = pd.DataFrame(columns=[by,"Recovery PSA Instantaneous Speed"])
    psa_peak_speed = pd.DataFrame(columns=[by,"Peak PSA Instantaneous Speed"])
    psa_peak_tap_speed = pd.DataFrame(columns=[by,"Peak Tap Number PSA Instantaneous Speed"])
    psa_avg_speed = pd.DataFrame(columns=[by,"Average PSA Instantaneous Speed"])
    psa_sensitization_speed = pd.DataFrame(columns=[by,"Sensitization PSA Instantaneous Speed"])
    psa_habituation_speed = pd.DataFrame(columns=[by,"Habituation PSA Instantaneous Speed"])
    psa_spontaneous_recovery_speed = pd.DataFrame(columns=[by,"Spontaneous Recovery PSA Instantaneous Speed"])
    psa_memory_retention_speed = pd.DataFrame(columns=[by,"Memory Retention PSA Instantaneous Speed"])
    # psa_initial_to_peak_speed = pd.DataFrame(columns=[by,"Initial_to_peak PSA Instantaneous Speed"])
    # psa_peak_to_recovery_speed = pd.DataFrame(columns=[by,"Peak_to_recovery PSA Instantaneous Speed"])

    psa_initial_bias = pd.DataFrame(columns=[by,"Initial PSA Bias"])
    psa_final_bias = pd.DataFrame(columns=[by,"Final PSA Bias"])
    psa_recovery_bias = pd.DataFrame(columns=[by,"Recovery PSA Bias"])
    psa_peak_bias = pd.DataFrame(columns=[by,"Peak PSA Bias"])
    psa_peak_tap_bias = pd.DataFrame(columns=[by,"Peak Tap Number PSA Bias"])
    psa_avg_bias = pd.DataFrame(columns=[by,"Average PSA Bias"])
    psa_sensitization_bias = pd.DataFrame(columns=[by,"Sensitization PSA Bias"])
    psa_habituation_bias = pd.DataFrame(columns=[by,"Habituation PSA Bias"])
    psa_spontaneous_recovery_bias = pd.DataFrame(columns=[by,"Spontaneous Recovery PSA Bias"])
    psa_memory_retention_bias = pd.DataFrame(columns=[by,"Memory Retention PSA Bias"])
    # psa_initial_to_peak_bias = pd.DataFrame(columns=[by,"Initial_to_peak PSA Bias"])
    # psa_peak_to_recovery_bias = pd.DataFrame(columns=[by,"Peak_to_recovery PSA Bias"])

    psa_initial_ang_speed = pd.DataFrame(columns=[by,"Initial PSA Angular Speed"])
    psa_final_ang_speed = pd.DataFrame(columns=[by,"Final PSA Angular Speed"])
    psa_recovery_ang_speed = pd.DataFrame(columns=[by,"Recovery PSA Angular Speed"])
    psa_peak_ang_speed = pd.DataFrame(columns=[by,"Peak PSA Angular Speed"])
    psa_peak_tap_ang_speed = pd.DataFrame(columns=[by,"Peak Tap Number PSA Angular Speed"])
    psa_avg_ang_speed = pd.DataFrame(columns=[by,"Average PSA Angular Speed"])
    psa_sensitization_ang_speed = pd.DataFrame(columns=[by,"Sensitization PSA Angular Speed"])
    psa_habituation_ang_speed = pd.DataFrame(columns=[by,"Habituation PSA Angular Speed"])
    psa_spontaneous_recovery_ang_speed = pd.DataFrame(columns=[by,"Spontaneous Recovery PSA Angular Speed"])
    psa_memory_retention_ang_speed = pd.DataFrame(columns=[by,"Memory Retention PSA Angular Speed"])
    # psa_initial_to_peak_ang_speed = pd.DataFrame(columns=[by,"Initial_to_peak PSA Angular Speed"])
    # psa_peak_to_recovery_ang_speed = pd.DataFrame(columns=[by,"Peak_to_recovery PSA Angular Speed"])

    psa_initial_aspect = pd.DataFrame(columns=[by,"Initial PSA Aspect Ratio"])
    psa_final_aspect = pd.DataFrame(columns=[by,"Final PSA Aspect Ratio"])
    psa_recovery_aspect = pd.DataFrame(columns=[by,"Recovery PSA Aspect Ratio"])
    psa_peak_aspect = pd.DataFrame(columns=[by,"Peak PSA Aspect Ratio"])
    psa_peak_tap_aspect = pd.DataFrame(columns=[by,"Peak Tap Number PSA Aspect Ratio"])
    psa_avg_aspect = pd.DataFrame(columns=[by,"Average PSA Aspect Ratio"])
    psa_sensitization_aspect = pd.DataFrame(columns=[by,"Sensitization PSA Aspect Ratio"])
    psa_habituation_aspect = pd.DataFrame(columns=[by,"Habituation PSA Aspect Ratio"])
    psa_spontaneous_recovery_aspect = pd.DataFrame(columns=[by,"Spontaneous Recovery PSA Aspect Ratio"])
    psa_memory_retention_aspect = pd.DataFrame(columns=[by,"Memory Retention PSA Aspect Ratio"])
    # psa_initial_to_peak_aspect = pd.DataFrame(columns=[by,"Initial_to_peak PSA Aspect Ratio"])
    # psa_peak_to_recovery_aspect = pd.DataFrame(columns=[by,"Peak_to_recovery PSA Aspect Ratio"])

    psa_initial_kink = pd.DataFrame(columns=[by,"Initial PSA Kink"])
    psa_final_kink = pd.DataFrame(columns=[by,"Final PSA Kink"])
    psa_recovery_kink = pd.DataFrame(columns=[by,"Recovery PSA Kink"])
    psa_peak_kink = pd.DataFrame(columns=[by,"Peak PSA Kink"])
    psa_peak_tap_kink = pd.DataFrame(columns=[by,"Peak Tap Number PSA Kink"])
    psa_avg_kink = pd.DataFrame(columns=[by,"Average PSA Kink"])
    psa_sensitization_kink = pd.DataFrame(columns=[by,"Sensitization PSA Kink"])
    psa_habituation_kink = pd.DataFrame(columns=[by,"Habituation PSA Kink"])
    psa_spontaneous_recovery_kink = pd.DataFrame(columns=[by,"Spontaneous Recovery PSA Kink"])
    psa_memory_retention_kink = pd.DataFrame(columns=[by,"Memory Retention PSA Kink"])
    # psa_initial_to_peak_kink = pd.DataFrame(columns=[by,"Initial_to_peak PSA Kink"])
    # psa_peak_to_recovery_kink = pd.DataFrame(columns=[by,"Peak_to_recovery PSA Kink"])

    psa_initial_curve = pd.DataFrame(columns=[by,"Initial PSA Curve"])
    psa_final_curve = pd.DataFrame(columns=[by,"Final PSA Curve"])
    psa_recovery_curve = pd.DataFrame(columns=[by,"Recovery PSA Curve"])
    psa_peak_curve = pd.DataFrame(columns=[by,"Peak PSA Curve"])
    psa_peak_tap_curve = pd.DataFrame(columns=[by,"Peak Tap Number PSA Curve"])
    psa_avg_curve = pd.DataFrame(columns=[by,"Average PSA Curve"])
    psa_sensitization_curve = pd.DataFrame(columns=[by,"Sensitization PSA Curve"])
    psa_habituation_curve = pd.DataFrame(columns=[by,"Habituation PSA Curve"])
    psa_spontaneous_recovery_curve = pd.DataFrame(columns=[by,"Spontaneous Recovery PSA Curve"])
    psa_memory_retention_curve = pd.DataFrame(columns=[by,"Memory Retention PSA Curve"])
    # psa_initial_to_peak_curve = pd.DataFrame(columns=[by,"Initial_to_peak PSA Curve"])
    # psa_peak_to_recovery_curve = pd.DataFrame(columns=[by,"Peak_to_recovery PSA Curve"])

    psa_initial_crab = pd.DataFrame(columns=[by,"Initial PSA Crab"])
    psa_final_crab = pd.DataFrame(columns=[by,"Final PSA Crab"])
    psa_recovery_crab = pd.DataFrame(columns=[by,"Recovery PSA Crab"])
    psa_peak_crab = pd.DataFrame(columns=[by,"Peak PSA Crab"])
    psa_peak_tap_crab = pd.DataFrame(columns=[by,"Peak Tap Number PSA Crab"])
    psa_avg_crab = pd.DataFrame(columns=[by,"Average PSA Crab"])
    psa_sensitization_crab = pd.DataFrame(columns=[by,"Sensitization PSA Crab"])
    psa_habituation_crab = pd.DataFrame(columns=[by,"Habituation PSA Crab"])
    psa_spontaneous_recovery_crab = pd.DataFrame(columns=[by,"Spontaneous Recovery PSA Crab"])
    psa_memory_retention_crab = pd.DataFrame(columns=[by,"Memory Retention PSA Crab"])
    # psa_initial_to_peak_crab = pd.DataFrame(columns=[by,"Initial_to_peak PSA Crab"])
    # psa_peak_to_recovery_crab = pd.DataFrame(columns=[by,"Peak_to_recovery PSA Crab"])

    list_psa_Tstats = [
        psa_initial_speed, psa_final_speed,psa_recovery_speed, psa_peak_speed, psa_peak_tap_speed, psa_avg_speed,
        psa_sensitization_speed, psa_habituation_speed, psa_spontaneous_recovery_speed, psa_memory_retention_speed,


        psa_initial_bias, psa_final_bias, psa_recovery_bias, psa_peak_bias, psa_peak_tap_bias, psa_avg_bias,
        psa_sensitization_bias, psa_habituation_bias, psa_spontaneous_recovery_bias, psa_memory_retention_bias,

        psa_initial_ang_speed, psa_final_ang_speed, psa_recovery_ang_speed, psa_peak_ang_speed, psa_peak_tap_ang_speed, psa_avg_ang_speed,
        psa_sensitization_ang_speed, psa_habituation_ang_speed, psa_spontaneous_recovery_ang_speed, psa_memory_retention_ang_speed,

        psa_initial_aspect, psa_final_aspect, psa_recovery_aspect, psa_peak_aspect, psa_peak_tap_aspect, psa_avg_aspect,
        psa_sensitization_aspect, psa_habituation_aspect, psa_spontaneous_recovery_aspect, psa_memory_retention_aspect,

        psa_initial_kink, psa_final_kink, psa_recovery_kink, psa_peak_kink, psa_peak_tap_kink, psa_avg_kink,
        psa_sensitization_kink, psa_habituation_kink, psa_spontaneous_recovery_kink, psa_memory_retention_kink,

        psa_initial_curve, psa_final_curve, psa_recovery_curve, psa_peak_curve, psa_peak_tap_curve, psa_avg_curve,
        psa_sensitization_curve, psa_habituation_curve, psa_spontaneous_recovery_curve, psa_memory_retention_curve,

        psa_initial_crab, psa_final_crab, psa_recovery_crab, psa_peak_crab, psa_peak_tap_crab, psa_avg_crab,
        psa_sensitization_crab, psa_habituation_crab, psa_spontaneous_recovery_crab, psa_memory_retention_crab
    ]

    list_psa_metrics = [
    "Initial PSA Instantaneous Speed",
    "Final PSA Instantaneous Speed",
    "Recovery PSA Instantaneous Speed",
    "Peak PSA Instantaneous Speed",
    "Peak Tap Number PSA Instantaneous Speed",
    "Average PSA Instantaneous Speed",
    "Sensitization PSA Instantaneous Speed",
    "Habituation PSA Instantaneous Speed",
    "Spontaneous Recovery PSA Instantaneous Speed",
    "Memory Retention PSA Instantaneous Speed",

    "Initial PSA Bias",
    "Final PSA Bias",
    "Recovery PSA Bias",
    "Peak PSA Bias",
    "Peak Tap Number PSA Bias",
    "Average PSA Bias",
    "Sensitization PSA Bias",
    "Habituation PSA Bias",
    "Spontaneous Recovery PSA Bias",
    "Memory Retention PSA Bias",

    "Initial PSA Angular Speed",
    "Final PSA Angular Speed",
    "Recovery PSA Angular Speed",
    "Peak PSA Angular Speed",
    "Peak Tap Number PSA Angular Speed",
    "Average PSA Angular Speed",
    "Sensitization PSA Angular Speed",
    "Habituation PSA Angular Speed",
    "Spontaneous Recovery PSA Angular Speed",
    "Memory Retention PSA Angular Speed",

    "Initial PSA Aspect Ratio",
    "Final PSA Aspect Ratio",
    "Recovery PSA Aspect Ratio",
    "Peak PSA Aspect Ratio",
    "Peak Tap Number PSA Aspect Ratio",
    "Average PSA Aspect Ratio",
    "Sensitization PSA Aspect Ratio",
    "Habituation PSA Aspect Ratio",
    "Spontaneous Recovery PSA Aspect Ratio",
    "Memory Retention PSA Aspect Ratio",


    "Initial PSA Kink",
    "Final PSA Kink",
    "Recovery PSA Kink",
    "Peak PSA Kink",
    "Peak Tap Number PSA Kink",
    "Average PSA Kink",
    "Sensitization PSA Kink",
    "Habituation PSA Kink",
    "Spontaneous Recovery PSA Kink",
    "Memory Retention PSA Kink",

    "Initial PSA Curve",
    "Final PSA Curve",
    "Recovery PSA Curve",
    "Peak PSA Curve",
    "Peak Tap Number PSA Curve",
    "Average PSA Curve",
    "Sensitization PSA Curve",
    "Habituation PSA Curve",
    "Spontaneous Recovery PSA Curve",
    "Memory Retention PSA Curve",

    "Initial PSA Crab",
    "Final PSA Crab",
    "Recovery PSA Crab",
    "Peak PSA Crab",
    "Peak Tap Number PSA Crab",
    "Average PSA Crab",
    "Sensitization PSA Crab",
    "Habituation PSA Crab",
    "Spontaneous Recovery PSA Crab",
    "Memory Retention PSA Crab"
]
    
    return list_psa_Tstats, list_psa_metrics


In [ ]:
def TTest(Type, DF_ref, output, by=["Gene", "dataset"]):
    """
    Perform two sample t-test for each unique Gene/dataset column in the Df_ref
    input: 
        - a:column name of values 
        - DF_ref:reference dataframe
        - output: output df to store results in 
        - by: what to group by "Gene" or "dataset"
        
    """
    for a in DF_ref[by].unique():
        Tstat_a = ttest_ind(DF_ref[DF_ref.dataset == a][Type], DF_ref[DF_ref.Allele.isin(["XJ1","N2"])][Type],equal_var=False)[0]
        Tstat_g = ttest_ind(DF_ref[DF_ref.Gene == a][Type], DF_ref[DF_ref.Gene == "N2"][Type],equal_var=False)[0]
        Tstat = Tstat_g if by=="Gene" else Tstat_a
        row = [a, Tstat]
        output.loc[len(output)] = row
    # print(output)

def do_TTest(by=["Gene", "dataset"], baseline=["true", "false", "psa"]):
    """
    Perform TTest function for each unique Gene/dataset column in baseline_output/tap_data
    
    input: 
        - by: what to group by "Gene" or "dataset"
        - baseline: whether or not to use baseline data

    returns: sorted T-statistics dataframe
    """

    if baseline=="true":
        list_Tstats, list_metrics = baseline_metrics(by)
        data = baseline_output
    elif baseline=="false":
        list_Tstats,list_metrics = tap_metrics(by)
        data = tap_data
    else:
        list_Tstats,list_metrics = psa_metrics(by)
        data = psa_data
    for x in data[by].unique():
        if Screen=="Neuron_Genes_Screen":
            condition = x in (["N2"] if by == "Gene" else ["N2_XJ1", "N2_N2"])
        else:
            condition = (x =="N2")
        if condition:
            pass
        else:
            output_gene=data[data[by]==x]
            gene_data=data[data['Date'].isin(output_gene['Date'].unique())]
            if Screen=="Neuron_Genes_Screen":
                gene_data_final = gene_data[gene_data[by].isin(['N2', x])] if by=="Gene" else gene_data[gene_data[by].isin(['N2_N2','N2_XJ1', x])]
            else:
                gene_data_final = gene_data[gene_data[by].isin(['N2', x])]

            for a,b in zip(list_metrics, list_Tstats):
                TTest(a, gene_data_final, b, by) # calls t test function
    
    PD_Tstats=pd.DataFrame()
    for a in list_Tstats:
        b=a.groupby([by], as_index=False).mean()
        if b.columns.values[1] == list_Tstats[0].columns.values[1]:
            PD_Tstats=b
        else:
            PD_Tstats=PD_Tstats.join(b.iloc[:,1])
            
    PD_Tstats=PD_Tstats.set_index(by)
    
    return PD_Tstats
            

### T-stat on Baseline data:

### 3.3.1 Allele-level T-stat analysis of baseline data

In [ ]:
warnings.filterwarnings('ignore')

PD_baseline_Tstats_allele = do_TTest("dataset", baseline="true") # get sorted T-statistics DataFrame 

# PD_baseline_Tstats_allele_sorted=PD_baseline_Tstats_allele.sort_index()

PD_baseline_Tstats_allele.head()

### 3.3.2 Gene-level T-stat analysis of baseline data

In [ ]:
warnings.filterwarnings('ignore')

PD_baseline_Tstats=do_TTest("Gene", baseline="true") # get sorted T-statistics DataFrame 

# PD_baseline_Tstats_sorted=PD_baseline_Tstats.sort_index()

PD_baseline_Tstats.head()

### T-stat analysis for tap-response data:

### 3.3.3 Allele level T-stat analysis of tap response data

In [ ]:
warnings.filterwarnings('ignore')

PD_habituation_Tstats_allele = do_TTest("dataset", baseline="false") # get sorted T-statistics DataFrame 

# PD_habituation_Tstats_allele_sorted=PD_habituation_Tstats_allele.sort_index()

PD_habituation_Tstats_allele.head()

### 3.3.4 Gene-level T-stat analysis of Tap response data

In [ ]:
warnings.filterwarnings('ignore')

PD_habituation_Tstats = do_TTest("Gene", baseline="false") # get sorted T-statistics DataFrame 

PD_habituation_Tstats_sorted=PD_habituation_Tstats.sort_index()

PD_habituation_Tstats.head()

### T-stat analysis for psa data:

### 3.3.5 Allele level T-stat analysis of PSA data

In [ ]:
warnings.filterwarnings('ignore')

psa_tstats_allele = do_TTest("dataset", baseline="psa") # get sorted T-statistics DataFrame 

psa_tstats_allele.head()

### 3.3.6 Gene-level T-stat analysis of PSA data

In [ ]:
warnings.filterwarnings('ignore')

psa_tstats = do_TTest("Gene", baseline="psa") # get sorted T-statistics DataFrame 

psa_tstats.head()

# 4. Merging t-stat data into one dataset

In [ ]:
def pop_cols(combined):
    """
    Reorders columns in the combined dataframe. 
    (pops specific columns["Area", "Midline", "Morphwidth", "Angular Speed"] and
    reinserts at different positions)

    input:
        combined: dataframe with columns to be reordered

    returns: 
        NA    
        
    """
    first_col=combined.pop("Area")
    combined.insert(0,"Area",first_col)

    first_col=combined.pop("Midline")
    combined.insert(0,"Midline",first_col)

    first_col=combined.pop("Morphwidth")
    combined.insert(0,"Morphwidth",first_col)

    first_col=combined.pop("Angular Speed")
    combined.insert(5,"Angular Speed",first_col)

def pop_last(combined):
    """
    Reorders the last three columns of the combined dataframe.
    input:
        combined: dataframe with columns to be reordered

    """
    last_col=combined.pop("Spontaneous Recovery of Response Duration")
    combined.insert(26,"Spontaneous Recovery of Response Duration",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Probability")
    combined.insert(26,"Spontaneous Recovery of Response Probability",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Speed")
    combined.insert(26,"Spontaneous Recovery of Response Speed",last_col)

    last_col=combined.pop("Memory Retention of Response Duration")
    combined.insert(26,"Memory Retention of Response Duration",last_col)

    last_col=combined.pop("Memory Retention of Response Probability")
    combined.insert(26,"Memory Retention of Response Probability",last_col)

    last_col=combined.pop("Memory Retention of Response Speed")
    combined.insert(26,"Memory Retention of Response Speed",last_col)

def rename_columns(df):
    '''
    Renames columns in the input dataframe
    input:
        combined: dataframe with columns to be renamed   
    returns:
        input dataframe with renamed columns 
    '''
    renames = {
        "Habituation of Duration": "Habituation of Response Duration",
        "Habituation of Probability": "Habituation of Respones Probability",
        "Habituation of Speed": "Habituation of Response Speed",
        "Initial Duration": "Initial Response Duration",
        "Initial Probability": "Initial Response Probability",
        "Initial Speed": "Initial Response Speed",
        "Final Duration": "Final Response Duration",
        "Final Probability": "Final Response Probability",
        "Final Speed": "Final Response Speed",
        "Recovery Duration": "Spontaneous Recovery of Response Duration",
        "Recovery Probability": "Spontaneous Recovery of Response Probability",
        "Recovery Speed": "Spontaneous Recovery of Response Speed",
        "Memory Retention Duration": "Memory Retention of Response Duration",
        "Memory Retention Probability": "Memory Retention of Response Probability",
        "Memory Retention Speed": "Memory Retention of Response Speed"
    }
    return df.rename(columns=renames)

def merge_Tstats(baseline, habituation, by=["Gene", "dataset"], Screen=Screen, psa=False):
    """
    merge baseline and tap response dataframes based on the Gene/dataset
    normalize the merged dataframe and then return it with melted version

    input:
        - baseline: baseline dataframe to merge
        - habituation: habituation dataframe to merge
        - by: what to group by "Gene" or "dataset"
    """

    #merge baseline and habituation data
    combined_Tstats = pd.merge(baseline, habituation, on=by, how='left')
    combined_Tstats = combined_Tstats.sort_index() # sort by index

    # ------------ NORMALISATION STEPS MOVED TO DASHBOARD -------------------
    # # normalise combined dataframe by subtracting mean and div by sd
    # combined_Tstats_normalized = (combined_Tstats-combined_Tstats.mean())/combined_Tstats.std()

    # if by=="dataset" and Screen=="Neuron_Genes_Screen":
    #     combined_Tstats_normalized_2 = combined_Tstats-combined_Tstats[combined_Tstats.index=="N2_XJ1"].squeeze()
    # else :
    #     combined_Tstats_normalized_2 = combined_Tstats-combined_Tstats[combined_Tstats.index=="N2"].squeeze()  

    pop_cols(combined_Tstats) # reorder columns

    # Skip this step if data = psa
    if not psa:
        #rename columns of combined and normalized df
        combined_Tstats = rename_columns(combined_Tstats)
        # combined_Tstats_normalized_2=rename_columns(combined_Tstats_normalized_2)
        pop_cols(combined_Tstats) # reorder columns
        pop_last(combined_Tstats) # reorder columns

    # -------------- PIVOTING STEPS MOVED TO DASHBOARD ---------------------
    # # Melt the combined dataframe
    # combined_Tstats_melted=combined_Tstats.reset_index()
    # combined_Tstats_melted=pd.melt(combined_Tstats_melted, id_vars=[by],
    #                             var_name='Metric',
    #                             value_name='T_score')
    
    # # Sort the melted dataframe by T_score
    # combined_Tstats_melted_sorted=combined_Tstats_melted.sort_values(by=['T_score'])

    # # Melt the normalized dataframe
    # combined_Tstats_normalized_melted=combined_Tstats.reset_index()
    # combined_Tstats_normalized_melted=pd.melt(combined_Tstats_normalized_melted, id_vars=[by],
    #                                                var_name='Metric',
    #                                                value_name='T_score')

    # add Screen column to df and its melted version
    combined_Tstats['Screen']=Screen
    # combined_Tstats_normalized_melted['Screen']=Screen

    return combined_Tstats#, combined_Tstats_normalized_melted



## 4.1 Gene-level

- Pass Tap and baseline through merge_Tstats() as df1
- Pass PSA and baseline through merge_Tstats()as df2
- pd.merge df1 and df2 using all columns of baseline

In [ ]:
# Baseline + Tap
combined_Tstats = merge_Tstats(PD_baseline_Tstats, PD_habituation_Tstats, "Gene")

In [ ]:
# Baseline + PSA 
combined_Tstats_psa = merge_Tstats(
    PD_baseline_Tstats, psa_tstats, by="Gene", psa=True
)

In [ ]:
# Baseline + Tap + PSA
final_tstat = pd.merge(combined_Tstats.reset_index(), combined_Tstats_psa.reset_index(), on = PD_baseline_Tstats.columns.to_list().append(['Gene','Screen']), how = 'inner')

final_tstat.head()

In [ ]:
# # Baseline + Tap + PSA melted
# final_tstat_melted = pd.concat([combined_Tstats_normalized_melted, combined_Tstats_psa_melted]).drop_duplicates()

# final_tstat_melted.head()

## 4.2 Allele level 


- Pass Tap and baseline through merge_Tstats() as df3
- Pass PSA and baseline through merge_Tstats()as df4
- pd.merge df3 and df4 using all columns of basline

In [ ]:
# Baseline + Tap
combined_Tstats_allele = merge_Tstats(PD_baseline_Tstats_allele,PD_habituation_Tstats_allele, "dataset")

In [ ]:
# Baseline + PSA 
combined_Tstats_psa_allele = merge_Tstats(
    PD_baseline_Tstats_allele, psa_tstats_allele, by="dataset", psa=True
)

In [ ]:
# Baseline + Tap + PSA
final_tstat_allele = pd.merge(combined_Tstats_allele.reset_index(), combined_Tstats_psa_allele.reset_index(), on = PD_baseline_Tstats_allele.columns.to_list().append(['dataset','Screen']), how = 'outer')

final_tstat_allele.head()

In [ ]:
final_tstat.shape

In [ ]:
# # Baseline + Tap + PSA melted
# final_tstat_melted_allele = pd.concat([combined_Tstats_normalized_melted_allele, combined_Tstats_psa_melted_allele]).drop_duplicates()

# final_tstat_melted_allele.head()

# 5. Save data to database (sqlite3)

#### A janky way to add data and update the sql 

1. Read table to pd.DataFrame
2. Add new data to pd.DataFrame
3. Replace old table with newly updated pd.DataFrame

# Primary Keys For Each SQL Table:

####  -- Gene_Allele_WormBaseID:
WBGene, WBAllele
#### -- alleleMSD:
dataset, Screen
#### -- gene_MSD:
Gene, Screen
#### -- allele_profile_data:
dataset, Metric, Screen
#### -- gene_profile_data:
Gene, Metric, Screen
#### -- tap_baseline_data:
Time, Plate_id, Date, Screen, dataset
#### -- tap_response_data:
plate, Date, Plate_id, Screen, taps, dataset, Gene, Allele
#### -- tstat_allele_data:
dataset, Screen
#### -- tstat_gene_data:
Gene, Screen
#### -- psa_summarized_data:
Plate_id,Date,Scree,dataset,Gene,Allele

In [ ]:
# print(tap_output.head(5))
# print(baseline_output.head(5))

tap_output.Screen = Screen
tap_psa_output.Screen = Screen
baseline_output.Screen = Screen

# print(tap_output.head(5))
# print(baseline_output.head(5))

In [ ]:
# final_tstat_allele[final_tstat_allele.isna().any(axis=1)]
final_tstat_allele[final_tstat_allele["Morphwidth"].isna()]

In [ ]:
# final_tstat_allele[final_tstat_allele['dataset'] == "unknown_CZ11000"]

In [ ]:

### This code will connect to PostgreSQL database and write non-duplicate data into the database tables.

# Loads database config values from database.ini file and validates that user and password are set.
config = load_config()
if (config['user'] == "" or config['password'] == ""):
    print("Please set your user and password in the database.ini file.")
    sys.exit(1)
    
# Creates a connection pool to PostgreSQL database using SQLAlchemy.
engine = create_engine(f"postgresql+psycopg://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")

# Function to insert data into PostgreSQL table, skipping duplicates based on primary keys.
def postgres_skip_on_duplicate(pd_table, conn, keys, data_iter):
    data = [dict(zip(keys,row)) for row in data_iter]
    conn.execute(insert(pd_table.table).on_conflict_do_nothing(), data)

# --------- Write the dataframes to PostgreSQL tables -----------

# Complete tap response data
print("working on tap_psa_output:") 
tap_psa_output.to_sql('tap_response_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)
# tap_psa_output.to_sql('tap_response_data', engine, if_exists='replace', index=False, method=None)

# Complete baseline data  >NO
print("working on tap_baseline_data:") 
baseline_output.to_sql('tap_baseline_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)
# baseline_output.to_sql('tap_baseline_data', engine, if_exists='replace', index=False, method=None)

# Baseline + Tap + PSA combined tstat data by Gene
print("working on tstat_gene_data")
final_tstat.dropna(thresh=10).reset_index().to_sql('tstat_gene_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)
# final_tstat.reset_index().to_sql('tstat_gene_data', engine, if_exists='replace', index=False, method=None)

# Baseline + Tap + PSA combined tstat data by Allele
print("working on tstat_allele_data")
final_tstat_allele.dropna(thresh=10).reset_index().to_sql('tstat_allele_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)
# final_tstat_allele.reset_index().to_sql('tstat_allele_data', engine, if_exists='replace', index=False, method=None)

# MSD Baseline + Tap + PSA by Gene
print("working on gene_MSD")
combined_MSD.to_sql('gene_MSD', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)
# combined_MSD.to_sql('gene_MSD', engine, if_exists='replace', index=False, method=None)

# MSD Baseline + Tap + PSA by Allele
print("working on allele_MSD")
allele_combined_MSD.to_sql('allele_MSD', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)
# allele_combined_MSD.to_sql('allele_MSD', engine, if_exists='replace', index=False, method=None)

# Summarised PSA data (speed, kink, curve, etc.)
print("working on psa_data:") 
psa_data.to_sql('psa_summarised_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)
# psa_data.to_sql('psa_summarised_data', engine, if_exists='replace', index=False, method=None)

# # Melted Baseline + Tap + PSA combined tstat data by Gene
# print("working on gene_profile_data")
# final_tstat_melted.to_sql('gene_profile_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

# # Melted Baseline + Tap + PSA combined tstat data by Allele
# print("working on allele_profile_data")
# final_tstat_melted_allele.to_sql('allele_profile_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)


print("---------- DONE ----------")

### Use the below cell if you want output to local .csv file

In [ ]:
# tap_psa_output.to_csv('/Users/Joseph/Desktop/PDScreen_tap_psa_output.csv', index=False)
# final_tstat.to_csv('/Users/Joseph/Desktop/PDScreen_final_tstat.csv', index=False)
# final_tstat_allele.to_csv('/Users/Joseph/Desktop/PDScreen_final_tstat_allele.csv', index=False)
# combined_MSD.to_csv('/Users/Joseph/Desktop/PDScreen_combined_MSD.csv', index=False)
# allele_combined_MSD.to_csv('/Users/Joseph/Desktop/PDScreen_combined_MSD_allele.csv', index=False)
# psa_data.to_csv('/Users/Joseph/Desktop/PDScreen_psa_data.csv', index=False)

### Use the below cell to just replace/update one table:

In [ ]:
# Loads database config values from database.ini file and validates that user and password are set.
config = load_config()
if (config['user'] == "" or config['password'] == ""):
    print("Please set your user and password in the database.ini file.")
    sys.exit(1)
    
# Creates a connection pool to PostgreSQL database using SQLAlchemy.
engine = create_engine(f"postgresql+psycopg://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")

# Function to insert data into PostgreSQL table, skipping duplicates based on primary keys.
def postgres_skip_on_duplicate(pd_table, conn, keys, data_iter):
    data = [dict(zip(keys,row)) for row in data_iter]
    conn.execute(insert(pd_table.table).on_conflict_do_nothing(), data)


# Complete tap response data
print("working on tap_output:") 
tap_psa_output.to_sql('tap_response_data', engine, if_exists='replace', index=False, method=None)
print("Done")

In [ ]:
# # USE THIS CELL TO UPDATE ALL THE NEED TALBES (Also have baseline_output on the second line)

# conn=sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/data_updated2.db')

# tap_output.to_sql('tap_response_data', conn, if_exists='append', index=False)

# baseline_output.to_sql('tap_baseline_data', conn, if_exists='append', index=False)

# combined_Tstats_normalize_2.reset_index().to_sql('tstat_gene_data', conn, if_exists='append', index=False)

# combined_Tstats_normalize_allele_2.reset_index().to_sql('tstat_allele_data', conn, if_exists='append', index=False)

# combined_Tstats_normalized_melted.to_sql('gene_profile_data', conn, if_exists='append', index=False)

# combined_Tstats_normalized_melted_allele.to_sql('allele_profile_data', conn, if_exists='append', index=False)

# combined_MSD.to_sql('gene_MSD', conn, if_exists='append', index=False)

# allele_combined_MSD.to_sql('allele_MSD', conn, if_exists='append', index=False)

# # combined_Tstats_melted_sorted.to_sql('allele_phenotype_data', conn, if_exists='replace', index=False)

# print(conn.total_changes)

# conn.close()


# # Want to test edge cases of pd.to_sql functionality#############